<h1>Processo de Treinamento do Ner-Drugs-Model</h1>

Etapas no processo de treinamento/utilização do <b>Ner-Drugs-Model</b>:
<ul>
    <li><b>Dataset</b></li>
    <li><b>Pré-processamento</b></li>
    <li><b>Config File</b></li>
    <li><b>Treinamento</b></li>
    <li><b>Implementação</b></li>
</ul>

<h1>O que é o Ner-Drugs-Model?</h1><br>
É um <b>spaCy model</b> (spaCy é uma biblioteca focada em processamento avançado de linguagem natural, ou NLP) treinado de maneira customizada para reconhecer palavras referentes a drogas na língua inglesa, como Oxycodone, Weed e Opium. 

<h1>Dataset</h1><br>
<p1>O <b>Dataset</b> utilizado no processo de treinamento e na avaliação do <b>Ner-Drugs-Model</b>, foi retirado do <a href="https://github.com/explosion/projects/tree/v3/tutorials/ner_drugs/assets">github</a> dos próprios developers da <b>spaCy</b> , onde esse Dataset é utilizado em um projeto de demonstração, esse dados foram retirados de comentários no reddit.<br>
    Dois arquivos foram baixados desse github, <b>drugs_train.jsonl</b>, utilizado para treinamento, e <b>drugs_eval.jsonl</b>, utilizado para avaliação.<br>
    Ambos arquivos estão em formato JSONL e cada data point contém o seguinte campos importantes:<br>
<ul>
    <li><b>text</b>: Contendo a string ou comentário.</li>
    <li><b>tokens</b>: Uma lista de dicionários com as palavras tokenizadas.</li>
    <li><b>spans</b>: Uma lista de dicionários que marca as entidades encontradas na string.</li>
</ul>
    Em seguida um exemplo de como é estruturado o Dataset:</p1><br><br>
<b><pre>{
  "text": "Idk if that Xanax or ur just an ass hole",
  "tokens": [
    { "text": "Idk", "start": 0, "end": 3, "id": 0 },
    { "text": "if", "start": 4, "end": 6, "id": 1 },
    { "text": "that", "start": 7, "end": 11, "id": 2 },
    { "text": "Xanax", "start": 12, "end": 17, "id": 3 },
    { "text": "or", "start": 18, "end": 20, "id": 4 },
    { "text": "ur", "start": 21, "end": 23, "id": 5 },
    { "text": "just", "start": 24, "end": 28, "id": 6 },
    { "text": "an", "start": 29, "end": 31, "id": 7 },
    { "text": "ass", "start": 32, "end": 35, "id": 8 },
    { "text": "hole", "start": 36, "end": 40, "id": 9 }
  ],
  "spans": [
    {
      "start": 12,
      "end": 17,
      "token_start": 3,
      "token_end": 3,
      "label": "DRUG"
    }
  ],
  "_input_hash": -2128862848,
  "_task_hash": -334208479,
  "answer": "accept"
}</pre></b><br><br>
Os <b>Datasets</b> anteriormente mencionados são compostos por centenas de linhas com objetos JSONL seguindo a estrutura descrita acima.


<h1>Pré-processamento</h1><br>
O <b>Pré-processamento</b> foi feito para converter o formato do <b>Dataset</b> de JSONL para <b>.spacy</b>, que é o formato padrão utilizado pela biblioteca <b>spaCy</b> para treinar os modelos.<br>
O Pré-processamento foi feito pelo seguinte código:

In [ ]:
import typer
import srsly
from pathlib import Path
from spacy.util import get_words_and_spaces
from spacy.tokens import Doc, DocBin
import spacy


def main(
    input_path: Path = typer.Argument(..., exists=True, dir_okay=False),
    output_path: Path = typer.Argument(..., dir_okay=False),
):
    nlp = spacy.blank("en")
    doc_bin = DocBin(attrs=["ENT_IOB", "ENT_TYPE"])
    for eg in srsly.read_jsonl(input_path):
        if eg["answer"] != "accept":
            continue
        tokens = [token["text"] for token in eg["tokens"]]
        words, spaces = get_words_and_spaces(tokens, eg["text"])
        doc = Doc(nlp.vocab, words=words, spaces=spaces)
        doc.ents = [
            doc.char_span(s["start"], s["end"], label=s["label"])
            for s in eg.get("spans", [])
        ]
        doc_bin.add(doc)
    doc_bin.to_disk(output_path)
    print(f"Processed {len(doc_bin)} documents: {output_path.name}")


if __name__ == "__main__":
    typer.run(main)

Com o <b>Pré-processamento</b> concluído, foram gerados dois arquivos, <b>drugs_eval.jsonl.spacy</b> e <b>drugs_train.jsonl.spacy</b>.

<h1>Config File</h1><br>
Em seguida a próxima etapa é o download do arquivo <b>base_config.cfg</b>, que pode ser obtido nesse <a href="https://spacy.io/usage/training">link</a>, esse arquivo base_config possui as informações necessárias básicas para se gerar um arquivo <b>config.ctg</b> definitivo, o qual será usado para treinar o modelo, mas antes o seguinte comando deve ser executado no terminal para gerar tal arquivo:<br><br>
<b>spacy init fill-config configs/base_config.cfg configs/config.ctg</b><br><br>
Consequentemente o arquivo <b>config.ctg</b> gerado é estruturado da seguinte forma: <br>
<pre>
    <b>
        [paths]
        train = "./corpus/drugs_train.jsonl.spacy"
        dev = "./corpus/drugs_eval.jsonl.spacy"
        vectors = null
        init_tok2vec = null

        [system]
        gpu_allocator = null
        seed = 0

        [nlp]
        lang = "en"
        pipeline = ["tok2vec","ner"]
        batch_size = 1000
        disabled = []
        before_creation = null
        after_creation = null
        after_pipeline_creation = null
        tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

        [components]

        [components.ner]
        factory = "ner"
        incorrect_spans_key = null
        moves = null
        update_with_oracle_cut_size = 100

        [components.ner.model]
        @architectures = "spacy.TransitionBasedParser.v2"
        state_type = "ner"
        extra_state_tokens = false
        hidden_width = 64
        maxout_pieces = 2
        use_upper = true
        nO = null

        [components.ner.model.tok2vec]
        @architectures = "spacy.Tok2VecListener.v1"
        width = ${components.tok2vec.model.encode.width}
        upstream = "*"

        [components.tok2vec]
        factory = "tok2vec"

        [components.tok2vec.model]
        @architectures = "spacy.Tok2Vec.v2"

        [components.tok2vec.model.embed]
        @architectures = "spacy.MultiHashEmbed.v2"
        width = ${components.tok2vec.model.encode.width}
        attrs = ["ORTH","SHAPE"]
        rows = [5000,2500]
        include_static_vectors = false

        [components.tok2vec.model.encode]
        @architectures = "spacy.MaxoutWindowEncoder.v2"
        width = 96
        depth = 4
        window_size = 1
        maxout_pieces = 3

        [corpora]

        [corpora.dev]
        @readers = "spacy.Corpus.v1"
        path = ${paths.dev}
        max_length = 0
        gold_preproc = false
        limit = 0
        augmenter = null

        [corpora.train]
        @readers = "spacy.Corpus.v1"
        path = ${paths.train}
        max_length = 0
        gold_preproc = false
        limit = 0
        augmenter = null

        [training]
        dev_corpus = "corpora.dev"
        train_corpus = "corpora.train"
        seed = ${system.seed}
        gpu_allocator = ${system.gpu_allocator}
        dropout = 0.1
        accumulate_gradient = 1
        patience = 1600
        max_epochs = 0
        max_steps = 20000
        eval_frequency = 200
        frozen_components = []
        annotating_components = []
        before_to_disk = null

        [training.batcher]
        @batchers = "spacy.batch_by_words.v1"
        discard_oversize = false
        tolerance = 0.2
        get_length = null

        [training.batcher.size]
        @schedules = "compounding.v1"
        start = 100
        stop = 1000
        compound = 1.001
        t = 0.0

        [training.logger]
        @loggers = "spacy.ConsoleLogger.v1"
        progress_bar = false

        [training.optimizer]
        @optimizers = "Adam.v1"
        beta1 = 0.9
        beta2 = 0.999
        L2_is_weight_decay = true
        L2 = 0.01
        grad_clip = 1.0
        use_averages = false
        eps = 0.00000001
        learn_rate = 0.001

        [training.score_weights]
        ents_f = 1.0
        ents_p = 0.0
        ents_r = 0.0
        ents_per_type = null

        [pretraining]

        [initialize]
        vectors = ${paths.vectors}
        init_tok2vec = ${paths.init_tok2vec}
        vocab_data = null
        lookups = null
        before_init = null
        after_init = null

        [initialize.components]

        [initialize.tokenizer]
    
</b></pre>

<h1>Treinamento</h1><br>
Nessa etapa todos os preparativos já foram feitos, só nós resta treinar o modelo, utilizando o seguinte comando:<br><br>
<b>spacy train ./configs/config.ctg --output ./output</b><br><br>
Dessa maneira o modelo está prontamente treinado e armazenado como <b>model-best</b> na pasta <b>output</b>, definida no comando anterior, e pode ser utilizado por meio do método <b>spacy.load()</b>.<br>
Só nós resta transformar tal modelo em uma <b>package</b> para que possa ser utilizado em qualquer python script, por meio do seguinte comando:<br><br>
<b>spacy package ./output/model-best ./model --name ner-drugs-model</b><br><br>
Agora o modelo pode ser facilmente importado.

<h1>Implementação</h1><br>
Agora na etapa final, uma pequena demonstração de como funciona, e é implementado, o Ner-Drugs-Model treinado.

In [9]:
import spacy
nlp = spacy.load("ner-drugs-model")
doc = nlp("Tell me more about weed, oxycodone and cocaine.")
for ent in doc.ents:
    print(ent.text,ent.label_)

weed DRUG
oxycodone DRUG
cocaine DRUG


Como pode ser visto acima o <b>Ner-Drugs-Model</b> reconheceu três <b>entidades</b> referentes a <b>label DRUG</b>.<br><br>
Assim concluindo a demonstração e o processo de treinamento.